In [ ]:
import os, glob, zipfile

# add Mount M

from google.colab import drive
drive.mount('/content/drive')


ROOT_DRIVE = "/content/drive/MyDrive/Seg_Dataset"
OUT_LOCAL  = "/content"   # write result zips to local
N, M = 10, 2               # choose how many video zips to include

train_zips = sorted(glob.glob(os.path.join(ROOT_DRIVE,"Train","*.zip")))[:N]
test_zips  = sorted(glob.glob(os.path.join(ROOT_DRIVE,"Test","*.zip")))[:M]

TRAIN_MINI_ZIP = f"{OUT_LOCAL}/SAR_RARP50_TRAIN_MINI_{N}.zip"
TEST_MINI_ZIP  = f"{OUT_LOCAL}/SAR_RARP50_TEST_MINI_{M}.zip"

def make_master(zip_list, dest):
    if os.path.exists(dest): os.remove(dest)
    with zipfile.ZipFile(dest, "w", compression=zipfile.ZIP_STORED, allowZip64=True) as zf:
        for p in zip_list:
            zf.write(p, arcname=os.path.basename(p))

make_master(train_zips, TRAIN_MINI_ZIP)
make_master(test_zips,  TEST_MINI_ZIP)

print("Created local zips:")
!ls -lh "$TRAIN_MINI_ZIP" "$TEST_MINI_ZIP"



In [ ]:
%%bash
set -e
TRAIN_MINI_ZIP="/content/SAR_RARP50_TRAIN_MINI_10.zip"
TEST_MINI_ZIP="/content/SAR_RARP50_TEST_MINI_2.zip"
OUT_TRAIN="/content/STSeg_train_10"
OUT_TEST="/content/STSeg_test_2"
N_CLASSES=9
PATCH="[512,640]"

rm -rf "$OUT_TRAIN" "$OUT_TEST"
mkdir -p "$OUT_TRAIN" "$OUT_TEST"

echo "[1/2] Converting TRAIN (10 videos) to Zarr..."
time stseg_create_dataset "$TRAIN_MINI_ZIP" "$OUT_TRAIN" $N_CLASSES "$PATCH"

echo "[2/2] Converting TEST (2 videos) to Zarr..."
time stseg_create_dataset "$TEST_MINI_ZIP" "$OUT_TEST"  $N_CLASSES "$PATCH"

echo
echo "[TRAIN Zarrs]"
ls -lh "$OUT_TRAIN/data" | sed -n '1,10p'
echo
echo "[TEST Zarrs]"
ls -lh "$OUT_TEST/data"  | sed -n '1,10p'


In [ ]:
import os, glob, json

DS = "/content/STSeg_train_10"
z = sorted(glob.glob(os.path.join(DS,"data","*.zarr")))
assert len(z) == 10, f"Expected 10 Zarrs, got {len(z)}"
ids = [os.path.splitext(os.path.basename(p))[0] for p in z]

split = {
  "train": ids[:8],
  "val":   ids[8:],
  "test":  []
}
sp = os.path.join(DS, "splits_train_val_test.json")
with open(sp, "w") as f: json.dump(split, f, indent=2)
print("Wrote:", sp)
print(open(sp).read())
